In [4]:
from gurobipy import Model, GRB

# Ensure correct dimensions for transport cost matrices
farms_transport_cost = farms.iloc[:, 5:5 + len(processing)].values  # Adjust to match the number of plants
processing_transport_cost = processing.iloc[:, 5:5 + len(centers)].values  # Adjust to match the number of centers

# Re-check dimensions
num_farms = len(farms)
num_plants = len(processing)
num_centers = len(centers)

# Model creation
model = Model("BioAgri_Optimization")

# Decision variables
x_fp = model.addVars(num_farms, num_plants, vtype=GRB.CONTINUOUS, name="x_fp")  # Raw material farm to plant
x_pc = model.addVars(num_plants, num_centers, vtype=GRB.CONTINUOUS, name="x_pc")  # Fertilizer plant to center

# Objective function: Minimize total cost
model.setObjective(
    sum(
        x_fp[f, p] * farms_transport_cost[f, p] for f in range(num_farms) for p in range(num_plants)
    ) +
    sum(
        x_pc[p, c] * (processing_cost[p] + processing_transport_cost[p, c])
        for p in range(num_plants) for c in range(num_centers)
    ),
    GRB.MINIMIZE
)

# Constraints
# 1. Farm capacity constraints
for f in range(num_farms):
    model.addConstr(sum(x_fp[f, p] for p in range(num_plants)) <= farms_capacity[f], name=f"Farm_Capacity_{f}")

# 2. Processing plant capacity constraints
for p in range(num_plants):
    model.addConstr(sum(x_fp[f, p] for f in range(num_farms)) <= processing_capacity[p], name=f"Plant_Capacity_Input_{p}")
    model.addConstr(sum(x_pc[p, c] for c in range(num_centers)) <= processing_capacity[p], name=f"Plant_Capacity_Output_{p}")

# 3. Demand fulfillment at each center
for c in range(num_centers):
    model.addConstr(sum(x_pc[p, c] for p in range(num_plants)) == centers_demand[c], name=f"Center_Demand_{c}")

# Optimize the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print(f"Minimum total cost: {model.objVal}")
    # Retrieve optimized decision variables
    farm_to_plant = {(f, p): x_fp[f, p].x for f in range(num_farms) for p in range(num_plants) if x_fp[f, p].x > 0}
    plant_to_center = {(p, c): x_pc[p, c].x for p in range(num_plants) for c in range(num_centers) if x_pc[p, c].x > 0}
    print("\nFarm to Plant Transportation:")
    print(farm_to_plant)
    print("\nPlant to Center Transportation:")
    print(plant_to_center)
else:
    print("Optimization was not successful.")


IndexError: index 17 is out of bounds for axis 1 with size 17